In [1]:
from site import check_enableusersite
from sodapy import Socrata
import pandas as pd
import numpy as np
import requests
from datetime import datetime

### Set up API

In [2]:
token = "dL93bWcHXE99lzC2Tyj7mJ4qR"
client = Socrata(
    "data.cityofchicago.org",
    token,
    timeout=1000000
)

### 2018 Rideshare

In [3]:
where2018 = "trip_start_timestamp<'2019-01-01T00:00:00.000' AND trip_start_timestamp>='2018-01-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"

# Count how many records there are in 2018
record_count2018 = client.get(
    "m6dm-c72p",
    select="COUNT(*)",
    # select="trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location",
    where=where2018,
)

In [4]:
# Record count of 2018
record_count2018

[{'COUNT': '15276428'}]

In [5]:
# Paging through 2018 data with 1,000,000 per chunk
# Takes about 1hr
start = 0
chunk_size = 1000000
results = []
while True:
    results.extend(
        client.get(
            "m6dm-c72p",
            select="trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location",
            where=where2018,
            offset=start,
            limit=chunk_size,
        )
    )
    print("chunk: " + str(start))
    start += chunk_size
    if start > int(record_count2018[0]["COUNT"]):
        break

rideshare2018 = pd.DataFrame.from_records(results)

chunk: 0
chunk: 1000000
chunk: 2000000
chunk: 3000000
chunk: 4000000
chunk: 5000000
chunk: 6000000
chunk: 7000000
chunk: 8000000
chunk: 9000000
chunk: 10000000
chunk: 11000000
chunk: 12000000
chunk: 13000000
chunk: 14000000
chunk: 15000000


In [6]:
# Change trip_start_timestamp to datetime variable
rideshare2018["trip_start_timestamp"] = pd.to_datetime(rideshare2018["trip_start_timestamp"])

# Create date variable which is only the date (remove specific time)
rideshare2018["date"] = pd.to_datetime(rideshare2018["trip_start_timestamp"]).dt.strftime("%Y-%m-%d")

# Group by date and randomly sample 0.1% of data
rideshare2018_reduced = rideshare2018.groupby("date").sample(frac=0.001)

# rideshare2018_reduced.to_csv("~/ids705/705-FinalProject/data/rideshare2018_reduced.csv")

### Jan 2019 - Jan 2020 Rideshare

In [7]:
# where_filter for each month
where2019jan = "trip_start_timestamp>='2019-01-01T00:00:00.000' AND trip_start_timestamp<'2019-02-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019feb = "trip_start_timestamp>='2019-02-01T00:00:00.000' AND trip_start_timestamp<'2019-03-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019mar = "trip_start_timestamp>='2019-03-01T00:00:00.000' AND trip_start_timestamp<'2019-04-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019apr = "trip_start_timestamp>='2019-04-01T00:00:00.000' AND trip_start_timestamp<'2019-05-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019may = "trip_start_timestamp>='2019-05-01T00:00:00.000' AND trip_start_timestamp<'2019-06-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019jun = "trip_start_timestamp>='2019-06-01T00:00:00.000' AND trip_start_timestamp<'2019-07-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019jul = "trip_start_timestamp>='2019-07-01T00:00:00.000' AND trip_start_timestamp<'2019-08-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019aug = "trip_start_timestamp>='2019-08-01T00:00:00.000' AND trip_start_timestamp<'2019-09-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019sep = "trip_start_timestamp>='2019-09-01T00:00:00.000' AND trip_start_timestamp<'2019-10-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019oct = "trip_start_timestamp>='2019-10-01T00:00:00.000' AND trip_start_timestamp<'2019-11-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019nov = "trip_start_timestamp>='2019-11-01T00:00:00.000' AND trip_start_timestamp<'2019-12-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2019dec = "trip_start_timestamp>='2019-12-01T00:00:00.000' AND trip_start_timestamp<'2020-01-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"
where2020jan = "trip_start_timestamp>='2020-01-01T00:00:00.000' AND trip_start_timestamp<'2020-02-01T00:00:00.000' AND pickup_community_area IS NOT NULL AND dropoff_community_area IS NOT NULL"

# Added to a list for convenience if running the function later in a loop (not advised, takes too long)
year2019_count = [
    where2019jan,
    where2019feb,
    where2019mar,
    where2019apr,
    where2019may,
    where2019jun,
    where2019jul,
    where2019aug,
    where2019sep,
    where2019oct,
    where2019nov,
    where2019dec,
    where2020jan,
]

# List of counts for each month
counts_bymonth = []
for month in year2019_count:
    count = client.get(
        "m6dm-c72p",
        select="COUNT(*)",
        where=month,
    )
    counts_bymonth.append(count)


In [8]:
def get_count(where_filter):
    """Count function. Returns in format of '[{'COUNT': 'XXXXXXX'}]'
    
    where_filter: filter condition for each month (string)
    """
    count = client.get(
        "m6dm-c72p",
        select="COUNT(*)",
        where=where_filter,
    )
    return count

In [9]:
def paging_data(where_filter, count):
    """Paging data with 1,000,000 per chunk.

    Play around with the chunk_size if 1 million is too large (roughly 500,000 - 1mil would be ideal).
    where_filter: filter condition (string).
    count: output from get_count function.
    
    Function returns a dataframe with selected month. 
    """
    start = 0
    chunk_size = 1000000
    results = []
    while True:
        results.extend(
            client.get(
                "m6dm-c72p",
                select="trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location",
                where=where_filter,
                offset=start,
                limit=chunk_size,
            )
        )
        start += chunk_size
        print("chunk: " + str(start))
        if start > int(count[0]["COUNT"]):
            break
    df = pd.DataFrame.from_records(results)
    return df

In [11]:
# Paging through data for each month
# Took 12hrs and did not finish so try not to use this
rideshare2019 = pd.DataFrame(columns=rideshare2018.columns)
for i in range(len(year2019_count)):
    dta = paging_data(year2019_count[i], counts_bymonth[i])
    rideshare2019 = pd.concat([rideshare2019, dta], axis=0)

KeyboardInterrupt: 

In [12]:
def reduce_data(df):
    """Takes output dataframe from paging_data and select 0.1% of data from every day.
    
    Returns a dataframe which can then be concatinated with other dataframes and returned
    as .csv. 
    """
    np.random.seed(1234)
    df["trip_start_timestamp"] = pd.to_datetime(df["trip_start_timestamp"])
    df["date"] = pd.to_datetime(df["trip_start_timestamp"]).dt.strftime("%Y-%m-%d")
    df_reduced = df.groupby("date").sample(frac=0.001)
    return df_reduced

### Example for paging through Jan 2019 data

In [ ]:
df2019jan = paging_data(where2019jan, get_count(where2019jan))
df2019jan_reduced = reduce_data(df2019jan)
df2019jan_reduced.to_csv("path")

In [13]:
df2019jul = paging_data(where2019jul, get_count(where2019jul))


chunk: 1000000
chunk: 2000000
chunk: 3000000
chunk: 4000000
chunk: 5000000
chunk: 6000000
chunk: 7000000
chunk: 8000000


In [14]:
df2019jul_reduced = reduce_data(df2019jul)


In [19]:
df2019jul_reduced.to_csv("jul.csv")

In [16]:
df2019aug = paging_data(where2019aug, get_count(where2019aug))


chunk: 1000000
chunk: 2000000
chunk: 3000000
chunk: 4000000
chunk: 5000000
chunk: 6000000
chunk: 7000000
chunk: 8000000
chunk: 9000000


In [17]:
df2019aug_reduced = reduce_data(df2019aug)


In [20]:
df2019aug_reduced.to_csv("aug.csv")

In [21]:
df2019sep = paging_data(where2019sep, get_count(where2019sep))


chunk: 1000000
chunk: 2000000
chunk: 3000000
chunk: 4000000
chunk: 5000000
chunk: 6000000
chunk: 7000000
chunk: 8000000


In [22]:
df2019sep_reduced = reduce_data(df2019sep)


In [23]:
df2019sep_reduced.to_csv("sep.csv")